In [99]:
from jupyter_dash import JupyterDash

In [4]:
from dash import Dash, dcc, html, Input, Output
import numpy as np
import plotly.express as px
from pyproj import Transformer
import plotly.graph_objects as go


x1 = -200
x2 = 400
h1 = 61
r1 = 10000


# Create forward rotation matrix    
theta = np.radians(29)
c, s = np.cos(theta), np.sin(theta)
rotation_matrix = np.array(((c, -s), (s, c)))

# Create backward rotation matrix
inv_theta = np.radians(-29)
c, s = np.cos(inv_theta), np.sin(inv_theta)
inv_rotation_matrix = np.array(((c, -s), (s, c)))

# Map dependend limits
xscale = 1. * 2000 / (0.48 + 0.35)
yscale = 1. * 2000 / (0.125 + 0.265)
x_min = -4 * xscale
x_max = 1.18 * xscale
y_min = -1.07 * yscale
y_max = 1 * yscale

# Create grid for isohypses
x_iso, y_iso = np.meshgrid(np.arange(x_min, x_max, 10), np.arange(y_min, y_max, 10))
iso_normal = np.stack([x_iso,y_iso], axis=2)

# Tilt grid for isohypses
iso_tilted= np.tensordot(inv_rotation_matrix, iso_normal, axes=([1],[2]))
x_iso_tilted = iso_tilted[0,:,:]
y_iso_tilted = iso_tilted[1,:,:]


lon_brunnen = 824550#13.71639
lat_brunnen = 5759000#51.88775

transformer = Transformer.from_crs("epsg:7416", "epsg:4326")


app = Dash(__name__)


app.layout = html.Div([
    html.H1('Trennstromlinie'),
    dcc.Graph(id="graph"),
    html.Label('Entnahmerate Q in l/s'),
    dcc.Slider(id="Q",
            min=0,
            max=1000,
            #marks={i: f"{i}" for i in np.arange(0., 1100., 100)},
            value=100,
        ),
    html.Label('Hydraulische Leitfähigkeit K in m/s'),
    dcc.Slider(id="K",
            min=0.0001,
            max=0.001,
            marks={i: f'{i}'for i in np.round(np.arange(0.0001, 0.0011, 0.0001), 6)},
            value=0.0004,
        ),

    html.Label('Hydraulischer Hintergrundgradient i in m/m'),
    dcc.Slider(id="grad",
            min=0,
            max=0.01,
            #marks={i: f'Label {i}' if i == 1 else str(i) for i in range(1, 6)},
            value=0.0017,
        )
])


def ymax_conf(Q, K, grad, b):
    ymax = Q/(2.*K*np.abs(grad)*b)
    return ymax

def x0_conf(Q, K, grad, b):
    x0 = -Q/(2.*np.pi*K*grad*b)
    return x0

def isohypsen_conf(h1, h2, x1, x2, K, Q, b, x, y):
    h_r1 = 0
    h_iso = (h2-h1)*(x-x1)/(x2-x1) +h1 + Q * np.log(np.sqrt(x**2+y**2) / r1) / (2 * np.pi * K * b) + h_r1
    return h_iso


def getXYtrennstromlinie(Q, K, grad):   
    # Umrechnung von l/s in m^3/s
    b = 120
    Q = Q / 1000
    
    h2 = h1 - grad * (x1 - x2)
    L = x1 - x2

    ymax = ymax_conf(Q, K, grad, b) * 0.95
    #x0 = x0_conf(Q, K, i, b)
    y = np.append(np.arange(-ymax, ymax, 10), ymax)
    x = y/(np.tan(2*np.pi*K*grad*b*y/Q))
    
    #for i_idx in x:
    #    print(i_idx)

    #mask = x > -30000
    #x = x[mask]
    #y = y[mask]
    
    
    h_iso = isohypsen_conf(h1, h2, x1, x2, K, Q, b, x_iso_tilted, y_iso_tilted)

    tsl_normal = np.vstack([x,y])
    tsl_tilted = rotation_matrix.dot(tsl_normal)
    
    final_x = tsl_tilted[0,:] + lon_brunnen
    final_y = tsl_tilted[1,:] + lat_brunnen
    
    # remove negatives
    mask = final_x >= 0
    final_x = final_x[mask]
    final_y = final_y[mask]
    
    
    tsl_normal = np.vstack([x,y])
    iso_normal = np.stack([x_iso,y_iso], axis=2)
    iso_tilted= np.tensordot(rotation_matrix, iso_normal, axes=([1],[2]))
    tsl_tilted = rotation_matrix.dot(tsl_normal)
    
    x_plot, y_plot = transformer.transform(final_x,final_y)

    return y_plot, x_plot


y_plot_init, x_plot_init = getXYtrennstromlinie(Q=100, K=0.0004, grad=0.0017)
fig = go.Figure(data=go.Scattermapbox(
    lon = y_plot_init,
    lat = x_plot_init,
    mode = 'lines',
    name = "Trennstromlinie"
    ))

brunnen_x_plot, brunnen_y_plot = transformer.transform(lon_brunnen,lat_brunnen)
fig.add_trace(
    go.Scattermapbox(
        lon = [brunnen_y_plot],
        lat = [brunnen_x_plot],
        mode="markers",
    marker = dict(
        size = 8),
        name = "Brunnen"
))
fig.update_mapboxes(style="open-street-map", 
               zoom = 11, 
               center={
               "lat":brunnen_x_plot, 
               "lon":brunnen_y_plot})
fig.update_layout(uirevision="Don't change"
    )

@app.callback(
    Output("graph", "figure"), 
    Input("Q", "value"),
    Input("K", "value"),
    Input("grad", "value"))
    #Input("b", "value"))
def TSL_conf(Q, K, grad):
    
    y_plot, x_plot = getXYtrennstromlinie(Q, K, grad)   
    fig.update_traces(lon = y_plot,
                    lat = x_plot,
                  selector=dict(mode="lines"))

    fig.show()
    return fig





app.run_server()



Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2022 18:04:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 18:04:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 18:04:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 18:04:04] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Jul/2022 18:04:04] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Jul/2022 18:04:04] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


127.0.0.1 - - [21/Jul/2022 18:04:05] "POST /_dash-update-component HTTP/1.1" 200 -


127.0.0.1 - - [21/Jul/2022 18:04:13] "POST /_dash-update-component HTTP/1.1" 200 -


127.0.0.1 - - [21/Jul/2022 18:04:15] "POST /_dash-update-component HTTP/1.1" 200 -


...

127.0.0.1 - - [21/Jul/2022 18:04:22] "POST /_dash-update-component HTTP/1.1" 200 -


127.0.0.1 - - [21/Jul/2022 18:04:23] "POST /_dash-update-component HTTP/1.1" 200 -


127.0.0.1 - - [21/Jul/2022 18:04:24] "POST /_dash-update-component HTTP/1.1" 200 -


127.0.0.1 - - [21/Jul/2022 18:04:27] "POST /_dash-update-component HTTP/1.1" 200 -


127.0.0.1 - - [21/Jul/2022 18:04:31] "POST /_dash-update-component HTTP/1.1" 200 -


In [98]:
import plotly.graph_objects as go
import numpy as np
import plotly.express as px



def ymax_conf(Q, K, i, b):
    ymax = Q/(2.*K*np.abs(i)*b)
    return ymax

def x0_conf(Q, K, i, b):
    x0 = -Q/(2.*np.pi*K*i*b)
    return x0



def isohypsen_conf(h1, h2, x1, x2, K, Q, b, x, y):
    h_r1 = 0
    h_iso = (h2-h1)*(x-x1)/(x2-x1) +h1 + Q * np.log(np.sqrt(x**2+y**2) / r1) / (2 * np.pi * K * b) + h_r1
    return h_iso

    
# Create forward rotation matrix    
theta = np.radians(29)
c, s = np.cos(theta), np.sin(theta)
rotation_matrix = np.array(((c, -s), (s, c)))

# Create backward rotation matrix
inv_theta = np.radians(-29)
c, s = np.cos(inv_theta), np.sin(inv_theta)
inv_rotation_matrix = np.array(((c, -s), (s, c)))

# Map dependend limits
xscale = 1. * 2000 / (0.48 + 0.35)
yscale = 1. * 2000 / (0.125 + 0.265)
x_min = -4 * xscale
x_max = 1.18 * xscale
y_min = -1.07 * yscale
y_max = 1 * yscale

# Create grid for isohypses
x_iso, y_iso = np.meshgrid(np.arange(x_min, x_max, 10), np.arange(y_min, y_max, 10))
iso_normal = np.stack([x_iso,y_iso], axis=2)

# Tilt grid for isohypses
iso_tilted= np.tensordot(inv_rotation_matrix, iso_normal, axes=([1],[2]))
x_iso_tilted = iso_tilted[0,:,:]
y_iso_tilted = iso_tilted[1,:,:]


#lon_brunnen = 13.71639
#lat_brunnen = 51.88775
lon_brunnen = 824550#13.71639
lat_brunnen = 5759000#51.88775

x1 = -200
x2 = 400
h1 = 61
r1 = 10000

Q=0.001
K=0.00001
i=0.0017
b=5
dist_iso=5

if True:
    h2 = h1 - i * (x1 - x2)
    L = x1 - x2

    ymax = ymax_conf(Q, K, i, b)
    #x0 = x0_conf(Q, K, i, b)
    y = np.arange(-ymax+0.1, ymax-0.1, 1)
    x = y/(np.tan(2*np.pi*K*i*b*y/Q))
    
    
    h_iso = isohypsen_conf(h1, h2, x1, x2, K, Q, b, x_iso_tilted, y_iso_tilted)

    tsl_normal = np.vstack([x,y])
    tsl_tilted = rotation_matrix.dot(tsl_normal)
    
    final_x = tsl_tilted[0,:] + lon_brunnen
    final_y = tsl_tilted[1,:] + lat_brunnen
    
    # remove negatives
    mask = final_x >= 0
    final_x = final_x[mask]
    final_y = final_y[mask]
    
    
    tsl_normal = np.vstack([x,y])
    iso_normal = np.stack([x_iso,y_iso], axis=2)
    iso_tilted= np.tensordot(rotation_matrix, iso_normal, axes=([1],[2]))
    tsl_tilted = rotation_matrix.dot(tsl_normal)
    
    from pyproj import Transformer
    transformer = Transformer.from_crs("epsg:7416", "epsg:4326")
    x_plot, y_plot = transformer.transform(final_x,final_y)
    brunnen_x_plot, brunnen_y_plot = transformer.transform(lon_brunnen,lat_brunnen)
    
    
    #fig = px.line_mapbox(lat=x_plot, lon=y_plot,#lat=final_y, lon=final_x, #hover_name=names,
    #                 mapbox_style="open-street-map", zoom=10)
    #fig.show()
    
    fig = go.Figure(data=go.Scattermapbox(
        lon = y_plot,
        lat = x_plot,
        mode = 'lines'
        ))
    fig.add_trace(
        go.Scattermapbox(
            lon = [brunnen_y_plot],
            lat = [brunnen_x_plot],
            mode="markers",
        marker = dict(
            size = 8)
    ))
    
    #fig.update_geos(
    #        center_lat=brunnen_x_plot,
    #        center_lon = brunnen_y_plot,
    #        #projection_type="mercator",
    #    projection_scale = 20
    #    #domain_x = [brunnen_y_plot - 10, brunnen_y_plot+10],
    #    #domain_y = [brunnen_x_plot - 10, brunnen_x_plot+10]
    #    )
    fig.update_mapboxes(style="open-street-map", 
                       zoom = 11, 
                       center={
                       "lat":brunnen_x_plot, 
                       "lon":brunnen_y_plot})

    #fig.add_trace(
    #px.scatter_mapbox(lat=[brunnen_x_plot], lon=[brunnen_y_plot])
    #)
    fig.show()

In [22]:
brunnen_y_plot

7.719143863355825

In [16]:
from pyproj import Proj, transform

inProj = Proj(init='epsg:25833')
outProj = Proj(init='epsg:4326')
x1,y1 = -11705274.6374,4826473.6922
x2,y2 = transform(inProj,outProj,final_x,final_y)


/home/grabow/anaconda3/envs/dash/lib/python3.10/site-packages/pyproj/crs/crs.py:130: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

/home/grabow/anaconda3/envs/dash/lib/python3.10/site-packages/pyproj/crs/crs.py:130: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

/tmp/ipykernel_8031/3935173023.py:6: DeprecationWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1



In [18]:
x2

array([8.2986102 , 8.50732682, 8.70076849, ..., 8.51628185, 8.31532708,
       8.09819524])

(array([49.66993616, 49.76137397, 49.84549663, ..., 49.8919741 ,
        49.80419977, 49.70862729]),
 array([2.2986102 , 2.50732682, 2.70076849, ..., 2.51628185, 2.31532708,
        2.09819524]))

In [86]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(
    go.Scattermapbox(
        lat=['45.5017'],
        lon=['-73.5673'],
        mode='markers',
        marker=dict(
            size=14
        ),
        text=['Montreal'],
        subplot='mapbox',
    ))
fig.add_trace(  
    go.Scattermapbox(
        lat=['45.5017'],
        lon=['-73.5673'],
        mode='markers',
        marker=dict(
            size=14
        ),
        text=['Montreal'],
        subplot='mapbox2',
    ))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        style='open-street-map',
        domain={'x': [0, 0.4], 'y': [0, 1]},
        bearing=0,
        center=dict(
            lat=45,
            lon=-73
        ),
        pitch=0,
        zoom=5
    ),
        mapbox2=dict(
        style='open-street-map',
        domain={'x': [0.6, 1.0], 'y': [0, 1]},
        bearing=0,
        center=dict(
            lat=45,
            lon=-73
        ),
        pitch=0,
        zoom=5
    ),
)


In [2]:
from dash import Dash, dcc, html, Input, Output
import numpy as np
import plotly.express as px
from pyproj import Transformer
import plotly.graph_objects as go


x1 = -200
x2 = 400
h1 = 61
r1 = 10000


# Create forward rotation matrix    
theta = np.radians(29)
c, s = np.cos(theta), np.sin(theta)
rotation_matrix = np.array(((c, -s), (s, c)))

# Create backward rotation matrix
inv_theta = np.radians(-29)
c, s = np.cos(inv_theta), np.sin(inv_theta)
inv_rotation_matrix = np.array(((c, -s), (s, c)))

# Map dependend limits
xscale = 1. * 2000 / (0.48 + 0.35)
yscale = 1. * 2000 / (0.125 + 0.265)
x_min = -4 * xscale
x_max = 1.18 * xscale
y_min = -1.07 * yscale
y_max = 1 * yscale

# Create grid for isohypses
x_iso, y_iso = np.meshgrid(np.arange(x_min, x_max, 10), np.arange(y_min, y_max, 10))
iso_normal = np.stack([x_iso,y_iso], axis=2)

# Tilt grid for isohypses
iso_tilted= np.tensordot(inv_rotation_matrix, iso_normal, axes=([1],[2]))
x_iso_tilted = iso_tilted[0,:,:]
y_iso_tilted = iso_tilted[1,:,:]


lon_brunnen = 824550#13.71639
lat_brunnen = 5759000#51.88775

transformer = Transformer.from_crs("epsg:7416", "epsg:4326")


app = Dash(__name__)


app.layout = html.Div([
    html.H1('Trennstromlinie'),
    dcc.Graph(id="graph"),
    html.Label('Entnahmerate Q in l/s'),
    dcc.Slider(id="Q",
            min=0,
            max=1000,
            marks={i: f"{i}" for i in np.arange(0., 1100., 100)},
            value=100,
        ),
    html.Label('Hydraulische Leitfähigkeit K in m/s'),
    dcc.Slider(id="K",
            min=0.0001,
            max=0.001,
            marks={i: f'{i}'for i in np.round(np.arange(0.0001, 0.0011, 0.0001), 6)},
            value=0.0004,
        ),

    html.Label('Hydraulischer Hintergrundgradient i in m/m'),
    dcc.Slider(id="i",
            min=0,
            max=99,
            #marks={i: f'Label {i}' if i == 1 else str(i) for i in range(1, 6)},
            value=0.0017,
        )
])


def ymax_conf(Q, K, i, b):
    ymax = Q/(2.*K*np.abs(i)*b)
    return ymax

def x0_conf(Q, K, i, b):
    x0 = -Q/(2.*np.pi*K*i*b)
    return x0

def isohypsen_conf(h1, h2, x1, x2, K, Q, b, x, y):
    h_r1 = 0
    h_iso = (h2-h1)*(x-x1)/(x2-x1) +h1 + Q * np.log(np.sqrt(x**2+y**2) / r1) / (2 * np.pi * K * b) + h_r1
    return h_iso


def getXYtrennstromlinie(Q, K, i):   
    # Umrechnung von l/s in m^3/s
    b = 120
    Q = Q / 1000
    
    h2 = h1 - i * (x1 - x2)
    L = x1 - x2

    ymax = ymax_conf(Q, K, i, b) * 0.95
    #x0 = x0_conf(Q, K, i, b)
    y = np.append(np.arange(-ymax, ymax, 10), ymax)
    x = y/(np.tan(2*np.pi*K*i*b*y/Q))
    
    #for i_idx in x:
    #    print(i_idx)

    #mask = x > -30000
    #x = x[mask]
    #y = y[mask]
    
    
    h_iso = isohypsen_conf(h1, h2, x1, x2, K, Q, b, x_iso_tilted, y_iso_tilted)

    tsl_normal = np.vstack([x,y])
    tsl_tilted = rotation_matrix.dot(tsl_normal)
    
    final_x = tsl_tilted[0,:] + lon_brunnen
    final_y = tsl_tilted[1,:] + lat_brunnen
    
    # remove negatives
    mask = final_x >= 0
    final_x = final_x[mask]
    final_y = final_y[mask]
    
    
    tsl_normal = np.vstack([x,y])
    iso_normal = np.stack([x_iso,y_iso], axis=2)
    iso_tilted= np.tensordot(rotation_matrix, iso_normal, axes=([1],[2]))
    tsl_tilted = rotation_matrix.dot(tsl_normal)
    
    x_plot, y_plot = transformer.transform(final_x,final_y)

    return y_plot, x_plot


y_plot_init, x_plot_init = getXYtrennstromlinie(Q=100, K=0.0004, i=0.0017)
fig = go.Figure(data=go.Scattermapbox(
    lon = y_plot_init,
    lat = x_plot_init,
    mode = 'lines',
    name = "Trennstromlinie"
    ))

brunnen_x_plot, brunnen_y_plot = transformer.transform(lon_brunnen,lat_brunnen)
fig.add_trace(
    go.Scattermapbox(
        lon = [brunnen_y_plot],
        lat = [brunnen_x_plot],
        mode="markers",
    marker = dict(
        size = 8),
        name = "Brunnen"
))
fig.update_mapboxes(style="open-street-map", 
               zoom = 11, 
               center={
               "lat":brunnen_x_plot, 
               "lon":brunnen_y_plot})
fig.update_layout(uirevision="Don't change"
    )

@app.callback(
    Output("graph", "figure"), 
    Input("Q", "value"),
    Input("K", "value"),
    Input("i", "value"))
    #Input("b", "value"))
def TSL_conf(Q, K, i):
    
    y_plot, x_plot = getXYtrennstromlinie(Q, K, i)   
    fig.update_traces(lon = y_plot,
                    lat = x_plot,
                  selector=dict(mode="lines"))

    fig.show()
    return fig



if False:
    K=6E-4
    i=0.0017
    dist_iso=1
    plot_coor=False
    continuous_update=False

    

if False:
    b = 100

    #layout = widgets.Layout(width='auto', height='40px')

    x,y = TSL_conf(0.0001, K, i, b, dist_iso, plot_coor)

    line = Polyline(
        locations=[[sub_y, sub_x] for sub_x, sub_y in zip(x,y)],
        color="green" ,
        fill=False
    )


app.run_server()



Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)


In [2]:
{i: f"{i}" for i in np.arange(0., 1100., 100)}

{0.0: '0.0',
 100.0: '100.0',
 200.0: '200.0',
 300.0: '300.0',
 400.0: '400.0',
 500.0: '500.0',
 600.0: '600.0',
 700.0: '700.0',
 800.0: '800.0',
 900.0: '900.0',
 1000.0: '1000.0'}